In [ ]:


Вы – единственный аналитик в компании, на которого легла ответственность за поиск инсайтов 
в данных продуктовых магазинов вашей компании.

В файле содержится информация о покупках людей

id  Товар Количество
1   Арбуз    1.0
1   Чай      1.0
1   Сгущёнка 0.5
2   Арбуз    3.0
2   Чай      1.0

id – означает покупку (в одну покупку входят все товары, купленные пользователем во время 1 похода в магазин)
Товар – наименование товара
Количество – число единиц купленного товара
Воспользуйтесь этими данными и выясните, какие пары товаров пользователи чаще всего покупают вместе. 
По сути, вам необходимо найти паттерны покупок, что позволит оптимизировать размещение продуктов в магазине, 
для удобства пользователей и увеличения выручки.

1_Товар    2_Товар     Встречаемость
  Чай      Арбуз            2
  Арбуз    Сгущёнка         1
  Чай      Сгущёнка         1
    
1_Товар – наименование первого товара
2_Товар – наименование второго товара
Встречаемость – число раз, когда такая пара была встречена
Другими словами: 2 раза люди покупали одновременно чай и арбуз, 1 раз одновременно покупали арбуз и сгущёнку 
и 1 раз одновременно были куплены чай со сгущёнкой.

Напишите код на python для получения нужной таблицы и укажите 5 наиболее распространённых паттернов.



In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import datetime
from matplotlib.ticker import FuncFormatter
import itertools
import operator
%matplotlib inline

In [2]:
#   Загрузим и посмотрим на данные

df = pd.read_csv('https://stepik.org/media/attachments/lesson/409319/test1_completed.csv')
df

,id,Товар,Количество
0,17119,Лимон,1.10
1,17119,Лимон оранжевый,0.70
2,17119,Лук-порей,10.00
3,17119,Лук репчатый,2.50
4,17119,Малина свежая,1.00
...,...,...,...
43509,119396,Укроп,0.25
43510,119396,Свекла,2.00
43511,119396,Физалис,1.00
43512,119396,Черешня сушеная,1.70


In [3]:
#   Сохраним в переменную food наши продукты.

food = df.Товар.unique()

In [4]:
#   Сгруппируем датафрейм по id наших покупок и укажем список товаров для каждой покупки.

df = df.groupby('id', as_index=False).agg({'Товар': list})
df

,id,Товар
0,17119,"[Лимон, Лимон оранжевый, Лук-порей, Лук репчат..."
1,17530,"[Лимон оранжевый, Изюм Султана, Капуста цветна..."
2,17618,"[Абрикосы молдавские, Кукуруза мини, Грибы лис..."
3,17724,"[Абрикосы молдавские, Арбуз, Салат Айсберг, Га..."
4,17814,"[Арбуз, Кабачки, Кинза, Малина свежая, Морковь..."
...,...,...
3268,119062,"[Баклажаны грунтовые, Персики, Персики армянск..."
3269,119110,"[Авокадо ХАСС, Апельсины столовые, Базилик зел..."
3270,119206,"[Огурцы длинноплодные, Грибы шампиньоны, Подпи..."
3271,119393,"[Петрушка, Грибы Еринги, Инжир свежий, Редис, ..."


In [5]:
#   А тут создадим список и сохраним в него всевозможные пары продуктов

arr = []

for subset in itertools.combinations(food, 2):
    arr.append(list(subset))

In [6]:
#   Настало время посчитать частоту встречаемости каждой пары продуктов.
#   Создадим словарь, в который будем сохранять в качестве ключа пару продуктов,
#   а в качестве значения - частоту встречаемости.

d = {}

for row in df.itertuples(name='pairs'):
    for pair in arr:
        if set(pair).issubset(row[2]):
            d[tuple(pair)] = d.get(tuple(pair), 0) + 1            

In [7]:
#   Функция для сортировки словаря по значениям по убыванию

def sort_dictionary_by_value(dictionary):
    list_of_sorted_pairs = [[list(k), dictionary[k]] for k in sorted(dictionary.keys(), key=dictionary.get, reverse=True)]
    # Так мы создаём кортежи (ключ, значение) из отсортированных элементов по ключу равному "значение ключа"
    # Также отсортированы будут и ключи, имеющие одно значение
    # "reverse = True" говорит, что сортировать нужно по убыванию
    return list_of_sorted_pairs 

In [8]:
#   Сохраним в новый словарь и посмотрим на ТОП-5 пар продуктов

new_d = sort_dictionary_by_value(d)

for x in range(5):
    print(new_d[x])

[['Огурцы Луховицкие', 'Укроп'], 431]
[['Петрушка', 'Укроп'], 408]
[['Огурцы Луховицкие', 'Арбуз'], 345]
[['Огурцы Луховицкие', 'Кабачки'], 326]
[['Укроп', 'Кинза'], 303]


In [9]:
#   Выглядит не очень удобно. Приведём к нормальному виду.


res = pd.DataFrame.from_dict(new_d)    #   Создадим датафрейм


def f_product_1(col):
    return col[0]
                                      #   Функции для выделения Товара_1 и Товара_2 в отдельные колонки

def f_product_2(col):
    return col[1]


res['Товар_1'] = res[0].apply(f_product_1)     #   Применяем функции
res['Товар_2'] = res[0].apply(f_product_2)

res = res.rename(columns={1: 'Встречаемость'})[['Товар_1', 'Товар_2', 'Встречаемость']]      #   Переименуем колонки

In [10]:
#   ТОП-5 пар продуктов

res.head()

,Товар_1,Товар_2,Встречаемость
0,Огурцы Луховицкие,Укроп,431
1,Петрушка,Укроп,408
2,Огурцы Луховицкие,Арбуз,345
3,Огурцы Луховицкие,Кабачки,326
4,Укроп,Кинза,303
